In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm


class MatrixFactorization():
    def __init__(self, R, k,  learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
    
    def fit(self):
        """
        self.b
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌
        """
        
        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])
        
        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            for i, j, in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))
            
            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print('Iteration: %d, cost : %.4f' % (epoch + 1, cost))
        
    def cost(self):
        """
        rmse를 계산하는 함수
        """
        
        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        # predicted = self.get_complete_matrix()
        cost = 0
        for x, y, in zip(xi, yi):
            cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
        return np.sqrt(cost/len(xi))
    
    def gradient(self, error, i, j):
        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq

    def gradient_descent(self, i, j, rating):
        prediction = self.get_prediction(i, j)
        error = rating - prediction
        
        # updata biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])
        
        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] = self._learning_rate * dp
        self._Q[j, :] = self._learning_rate * dq
        
    def get_prediction(self, i, j):
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)
    
    def get_complete_matrix(self):
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)

In [2]:
# run example

# rating matrix - Uset - Item : (7 x 5)
R = np.array([
    [1, 0, 0, 1, 3],
    [2, 0, 3, 1, 1],
    [1, 2, 0, 5, 0],
    [1, 0, 0, 4, 4],
    [2, 1, 5, 4, 0],
    [5, 1, 5, 4, 0],
    [0, 0, 0, 1, 0],
])

# P, Q is (7 x k), (k x 5) matrix

In [3]:
%%time
factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()

Iteration: 10, cost : 1.3480
Iteration: 20, cost : 1.1981
Iteration: 30, cost : 1.1007
Iteration: 40, cost : 1.0354
Iteration: 50, cost : 0.9902
Iteration: 60, cost : 0.9584
Iteration: 70, cost : 0.9354
Iteration: 80, cost : 0.9187
Iteration: 90, cost : 0.9062
Iteration: 100, cost : 0.8968
CPU times: user 108 ms, sys: 104 ms, total: 212 ms
Wall time: 85.1 ms


In [4]:
factorizer.get_complete_matrix()

array([[ 1.00586572, -0.1436662 ,  3.16756266,  2.01667428,  2.23407583],
       [ 0.71656845, -0.43296347,  2.87826539,  1.72737701,  1.94477856],
       [ 2.56780519,  1.41827326,  4.72950213,  3.57861374,  3.7960153 ],
       [ 2.26915661,  1.11962469,  4.43085355,  3.27996517,  3.49736672],
       [ 2.44274145,  1.29320953,  4.60443839,  3.45355001,  3.67095156],
       [ 3.16058776,  2.01105583,  5.3222847 ,  4.17139631,  4.38879787],
       [ 0.70760456, -0.44192737,  2.8693015 ,  1.71841311,  1.93581467]])

### 코드의 Step by Step 설명 

기본적인 구조는 위와 같다
```python
class MatrixFactorization():
    def __init__(self, R, k,  learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
```
일단, init 함수는 Matrix Factorization이라는 클래스가 호출될 때 자동으로 실행되는 부분입니다. 파라미터로는 6개의 인자를 받는데 각각 아래의 의미를 가진다

- R: 평점 행렬
- k: User Latent와 Item Latent의 차원의 수
- learning_rate: 학습률
- reg_param: Weight의 Regularization 값
- epochs: 전체 학습 횟수 (Total Epoch)
- verbose: 학습 과정을 출력할지 여부 (True : 10번마다 cost 출력, False : cost를 출력하지 않음)

```python
%%time

R = np.array([
    [1, 0, 0, 1, 3],
    [2, 0, 3, 1, 1],
    [1, 2, 0, 5, 0],
    [1, 0, 0, 4, 4],
    [2, 1, 5, 4, 0],
    [5, 1, 5, 4, 0],
    [0, 0, 0, 1, 0],
])

factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True)
factorizer.fit()
```
위의 실행은 7개의 User, 5개의 Item에 대한 평점 행렬(R)에 대해서 k=3, learning_rate=0.01, reg_param=0.01, epochs=100, verbose=True와 같은 파라미터를 가지는 MatrixFactorization 객체를 생성하라는 의미이다   
생성된 객체에서 factorizer.fit()을 통해서 fit()함수를 실행하면, 아래의 코드가 실행되게 된다

```python
def fit(self):
    """
    self.b
    - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
    - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌
    """

    # init latent features
    self._P = np.random.normal(size=(self._num_users, self._k))
    self._Q = np.random.normal(size=(self._num_items, self._k))

    # init biases
    self._b_P = np.zeros(self._num_users)
    self._b_Q = np.zeros(self._num_items)
    self._b = np.mean(self._R[np.where(self._R != 0)])

    # train while epochs
    self._training_process = []
    for epoch in range(self._epochs):
        # rating이 존재하는 index를 기준으로 training
        xi, yi = self._R.nonzero()
        for i, j, in zip(xi, yi):
            self.gradient_descent(i, j, self._R[i, j])
        cost = self.cost()
        self._training_process.append((epoch, cost))

        # print status
        if self._verbose == True and ((epoch + 1) % 10 == 0):
            print('Iteration: %d, cost : %.4f' % (epoch + 1, cost))
```
fit 함수에서 가장 먼저 실행되는 부분은 아래의 Latent matrix를 초기화 해주는 부분이다

```python
# init latent features
self._P = np.random.normal(size=(self._num_users, self._k))
self._Q = np.random.normal(size=(self._num_items, self._k))

# init biases
self._b_P = np.zeros(self._num_users)
self._b_Q = np.zeros(self._num_items)
self._b = np.mean(self._R[np.where(self._R != 0)])
```

이 때, np.random.normal함수를 이용해서 행렬을 초기화해주고 np.zeros 함수를 이용해서 bias부분을 초기화해준다 함수의 의미는 아래와 같다
- np.random.normal : (self._num_users, self._k)의 크기로 행렬을 정규분포 형태로 초기화합니다 위의 예시에서는 User Latent Matrix는 (7, 3)의 크기를, Item Latent Matrix는 (5, 3)의 크기를 가짐
- np.zeros : self._num_users 혹은 self._num_items의 크기만큼 0 값을 가지는 벡터를 생성합니다
- np.mean(self._R[np.where(self._R != 0)]) : 전체 평점의 평균을 계산     

이후 전체 학습 과정을 진행

```python
# train while epochs
self._training_process = []
for epoch in range(self._epochs):
    # rating이 존재하는 index를 기준으로 training
    xi, yi = self._R.nonzero()
    for i, j, in zip(xi, yi):
        self.gradient_descent(i, j, self._R[i, j])
    cost = self.cost()
    self._training_process.append((epoch, cost))

    # print status
    if self._verbose == True and ((epoch + 1) % 10 == 0):
        print('Iteration: %d, cost : %.4f' % (epoch + 1, cost))
```
- self._training_process = [] 는 for 문 안의 self._training_process.append((epochs, cost)) 에 사용되는 부분으로 학습 시에 Epoch와 cost를 저장하는 부분이다 for문의 경우 처음 파라미터로 받은 self._epochs 만큼 반복학습을 진행한다
- 먼저 시행되는 fora 문의 첫 번째로 시행되는 xi, yi = self._R.nonzero()는 평점 행렬에서 0이 아닌 부분 즉, 사용자가 평점을 매긴 부분에 대해서만 값을 추출하라는 의미이다 그 이유는 위의 이론에서 말했듯이 결측치가 아닌 부분을 통해서만 학습을 진행 하려는 의도이다 이후에 해당 부분을 통해서 모든 평점 부분에 대해서 gradient_descent를 실행한다

```python
for i, j in zip(xi, yi):
    self.gradient_descent(i, j, self._R[i, j])
```
     
     
```python
def gradient_descent(self, i, j, rating):
    prediction = self.get_prediction(i, j)
    error = rating - prediction

    # updata biases
    self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
    self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

    # update latent feature
    dp, dq = self.gradient(error, i, j)
    self._P[i, :] = self._learning_rate * dp
    self._Q[j, :] = self._learning_rate * dq
```
- gradient_descent 함수의 경우 행렬의 원소 위치(i, j)와 평점 값(self.R[i, j]을 받는다 바로 시작하는 prediction = self.get_prediction(i, j)은 User Latent Matrix와 Item Latent Matrix의 곱을 통해서 평점 행렬의 값들을 생성하는 부분이다
- self._P[i, :].dot(self._Q[j, :].T) 에서 User Latent P와 Item Latent Q가 곱해져서 평점을 계산하고 Bias를 없애기 위해서 전체 평균(self._b), User의 평균 평점(self._b_P[i]), Item의 평균 평점(self._b_Q[j])을 더해줌으로써 값을 생성한다

```python
def get_prediction(self, i, j):
    return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)
```

이후에, error = rating - prediction 을 통해서 얼마만큼의 차이가 있는지 계산을 합니다 그리고 아래의 수식에서 사용한 공식을 이용해서 self.gradient(error, i, j) 에서 Gradient값을 계산하고 Weight 및 Bias를 업데이트한다

![](https://github.com/J-TKim/T_Academy_Recommendation/blob/main/image/SGD_formula.png?raw=true "수식")
```python
# updata biases
self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

# update latent feature
dp, dq = self.gradient(error, i, j)
self._P[i, :] = self._learning_rate * dp
self._Q[j, :] = self._learning_rate * dq
```

이후에, cost = self.cost()에서 전체 Matrix에 대해서 오차를 계산하고 출력해준다 cost += pow(Self._R[x, y] - self.get_prediciton(x, y), 2) 에서 각 평점 별로 오차를 계산하게 된다
```python
def cost(self):
    """
    rmse를 계산하는 함수
    """

    # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다
    # 참고: http://codepractice.tistory.com/90
    xi, yi = self._R.nonzero()
    # predicted = self.get_complete_matrix()
    cost = 0
    for x, y, in zip(xi, yi):
        cost += pow(self._R[x, y] - self.get_prediction(x, y), 2)
    return np.sqrt(cost/len(xi))
```

이후, 모든 Epoch에 대해서 학습이 완료되면 factorizer.get_complete_matrix()를 통해서 완성된 평점 행렬을 추출하게 되면 SGD를 이용한 협업 필터링이 완료되게 된다